# AlexNet achievement and modify

Reference: 

Paper: http://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf

Web:  https://kratzert.github.io/2017/02/24/finetuning-alexnet-with-tensorflow.html


AlexNet结构：

|Layer|Type|Channels|Size|Kernel size| Stride|Padding|Activation|Split|
|-|-|-|-|-|-|-|-|-|
|Out|FC|-|1,000|-|-|-|Softmax|N/A|
|F9|FC|-|4096|-|-|-|ReLU|False|
|F8|FC|-|4096|-|-|-|ReLU|False|
|C7|Conv|256|13*13|3*3|1|SAME|ReLU|True|
|C6|Conv|384|13*13|3*3|1|SAME|ReLU|True|
|C5|Conv|384|13*13|3*3|1|SAME|ReLU|Fasle|
|P4|MaxPooling|256|13*13|3*3|2|VALID|-|True|
|C3|Conv|256|27*27|5*5|1|SAME|ReLU|Ture|
|P2|MaxPooling|96|27*27|3*3|2|VALID|-|True|
|C1|Conv|96|55*55|11*11|4|SAME|ReLU|True|
|In|Input|3|224*224|-|-|-|-|N/A|

代码解析：
网络构造create()
```python
def create():
    """
    8th Layer: FC and return unscaled activations
    7th Layer: FC (w ReLu) -> Dropout
    6th Layer: Flatten -> FC (w ReLu) -> Dropout
    5th Layer: Conv (w ReLu) -> Pool splitted into two groups
    4th Layer: Conv (w ReLu) splitted into two groups
    3rd Layer: Conv (w ReLu)
    2nd Layer: Conv (w ReLu)  -> Lrn -> Pool with 2 groups
    1st Layer: Conv (w ReLu) -> Lrn -> Pool
    """
    
    # 1st Layer: Conv (w ReLu) -> Lrn -> Pool
    conv1 = conv(self.X, 11, 11, 96, 4, 4, padding='VALID', name='conv1')
    norm1 = lrn(conv1, 2, 1e-04, 0.75, name='norm1')
    pool1 = max_pool(norm1, 3, 3, 2, 2, padding='VALID', name='pool1')

    # 2nd Layer: Conv (w ReLu)  -> Lrn -> Pool with 2 groups
    conv2 = conv(pool1, 5, 5, 256, 1, 1, groups=2, name='conv2')
    norm2 = lrn(conv2, 2, 1e-04, 0.75, name='norm2')
    pool2 = max_pool(norm2, 3, 3, 2, 2, padding='VALID', name='pool2')

    # 3rd Layer: Conv (w ReLu)
    conv3 = conv(pool2, 3, 3, 384, 1, 1, name='conv3')

    # 4th Layer: Conv (w ReLu) splitted into two groups
    conv4 = conv(conv3, 3, 3, 384, 1, 1, groups=2, name='conv4')

    # 5th Layer: Conv (w ReLu) -> Pool splitted into two groups
    conv5 = conv(conv4, 3, 3, 256, 1, 1, groups=2, name='conv5')
    pool5 = max_pool(conv5, 3, 3, 2, 2, padding='VALID', name='pool5')

    # 6th Layer: Flatten -> FC (w ReLu) -> Dropout
    flattened = tf.reshape(pool5, [-1, 6*6*256])
    fc6 = fc(flattened, 6*6*256, 4096, name='fc6')
    dropout6 = dropout(fc6, self.KEEP_PROB)

    # 7th Layer: FC (w ReLu) -> Dropout
    fc7 = fc(dropout6, 4096, 4096, name='fc7')
    dropout7 = dropout(fc7, self.KEEP_PROB)

    # 8th Layer: FC and return unscaled activations
    self.fc8 = fc(dropout7, 4096, self.NUM_CLASSES, relu=False, name='fc8')

```

基本操作： 卷积，全连接，池化，LRN，Dropout
```python
def conv(arguments):
    pass
def fc(arguments):
    pass
def max_pool(arguments):
    pass
def lrn(arguments):
    pass
def dropout(argments):
    pass:
```

### tensorflow 数据流式 读取方式